In [ ]:
from IPython import display
display.Image('/Users/vamsi/DESKTOP/KPMG/image.png', width=1200)

## KPMG Internship Module_1

**Project Name: "Customers Recommendation Project"**  
**Client: "Sprocket Central Pty Ltd Company"**

**Project Brief:**  
Sprocket Central Pty Ltd, a medium-sized organization specializing in bikes and cycling accessories, has provided KPMG with three datasets: customer demographic, customer addresses, and transaction data for the past three months. The client needs help analyzing this data to optimize their marketing strategy for the new customer list.

**Module#01 Objective: Data Quality Assessment Report.**  
The primary objective of this report is to review the data quality of all three datasets and ensure their readiness for analysis. To achieve this, we will carry out separate wrangling, cleaning and feature engineering procedures for each dataset before merging them into a single dataset.

**Note:** additionally, we have also assessed the new customer list, which consists of 1000 customers, to ensure its readiness for module #03.

## Table of Contents

- [1.0. Transactions dataset](#transactions-dataset)
  - [1.1. Data Quality Assessment](#transactions-dataset-data-quality-assessment)
    - [Intital Examination]
    - [Intital Observations]
    - [Features Engineering]
    - [Missing Data]
    - [Addressing Data Quality Issues]
    - [Creating Additional Variables]
- [2.0. Customer_demographic dataset](#customer-demographic-dataset)
  - [2.1. Data Quality Assessment](#customer-demographic-dataset-data-quality-assessment)
    - [Intital Examination]
    - [Intital Observations]
    - [Features Engineering]
    - [Missing Data]
    - [Addressing Data Quality Issues]
    - [Creating Additional Variables]
- [3.0. Customer_address dataset](#customer-address-dataset)
  - [3.1. Data Quality Assessment](#customer-address-dataset-data-quality-assessment)
    - [Intital Examination]
    - [Intital Observations]
    - [Features Engineering]
    - [Missing Data]
    - [Addressing Data Quality Issues]
- [4.0. Data Integration](#data-integration)
- [5.0. New customers dataset](#new-customers-dataset)
  - [5.1. Data Quality Assessment](#new-customers-dataset-data-quality-assessment)
    - [Intital Examination]
    - [Intital Observations]
    - [Features Engineering]
    - [Missing Data]
    - [Addressing Data Quality Issues]


In [ ]:
# import libraries
import pandas as pd 
from pandas_profiling import ProfileReport
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import calendar 
import datetime as dt  
%matplotlib inline 

----------------------------------------------------------------------------------------------------------------------

## 1.0. Transactions dataset<a id='transactions-dataset'></a>

In [ ]:
# read in transactions sheet from file
transactions = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='Transactions', header=1)

### 1.1. Data Quality Assessment:<a id='transactions-dataset-data-quality-assessment'></a>
**Intital Examination:**

In [ ]:
# check first few rows
transactions.head()

In [ ]:
# check summary statistics
transactions.info()

In [ ]:
# check descriptive statistics 
transactions.describe()

In [ ]:
# generate a detailed report for "transactions" data set using pandas-profiling
transactions_profile = ProfileReport(transactions)
transactions_profile

**Intital Observations:**

After examining the dataframe, the identified issues are:

**'list_price' and 'standard_cost'** columns are not in numerical format, potentially affecting data analysis.    
**'transaction_date'** column needs to be converted to a datetime format for better data handling.    
Separate columns for **year, month name, day, and day name** are useful for temporal analysis.    
Multiple columns have missing values that need addressing before analysis: **'online_order', 'brand', 'product_line', 'product_class', 'product_size', 'standard_cost', and 'product_first_sold_date'**  

The quantity of transactions for product with id zero is extremely larger than the rest of the products.  
This could be due to an error (products with id "zero" are products with missing values for id), as there are no null values in the product_id column.    
Another possibility is that there is a correlation between certain variables such as cost and listing price, product size and cost, and brand and listing price.    
Additionally, there are 360 orders that are unknown in sale source (online/offline).    

**Features Engineering:**

The following strategies can be used to handle missing data:  

**Fill missing values with the mean** for continuous variables without outliers.  
**Fill missing values with the median** for continuous variables with outliers.  
**Fill missing values with the mode** for categorical variables.  
**Use forward-fill (ffill) or backward-fill (bfill)** methods to fill missing data.  
**Drop the entire variable** if there are a large number of missing data points.  
**Drop rows containing null values** if they won't significantly impact the analysis.  

In [ ]:
# count missing values in each variable
transactions.isnull().sum()

In [ ]:
# calculate missing value percentage for each column
transactions.isnull().mean() * 100

**Missing Data:** 

Steps to address missing data:

**'online_order'** has only 1.8% missing data, which will be imputed using the mode, the most common data point in the dataset.    
**'brand', 'product_line', 'product_class', 'product_size', 'standard_cost', and 'product_first_sold_date'** have less than 1% missing data and share the same missing data pattern. Thus, we will remove the rows containing null values for these variables. This step is justified since the percentage of missing data is relatively low, and removing these records is unlikely to impact the validity of our analysis.  

In [ ]:
# drop column if too many missing datapoints
drop_threshold = transactions.shape[0]*0.5
transactions = transactions.dropna(thresh=drop_threshold, how='all', axis='columns').copy()

In [ ]:
# count missing values in 'online_order' column
transactions['online_order'].isnull().sum()

In [ ]:
# count number of online orders vs. offline orders
transactions['online_order'].value_counts()

In [ ]:
# fill missing values in the 'online_order' column with the mode value
mode_online_order = transactions['online_order'].mode()[0]
transactions['online_order'] = transactions['online_order'].fillna(mode_online_order)
transactions['online_order'].value_counts()

In [ ]:
# remove rows with null values in other columns
transactions.dropna(axis=0,inplace=True)

In [ ]:
# check results after removing null values
transactions.isnull().sum()

**Addressing Data Quality Issues:**

To ensure data quality, we will examine the dataframe for several aspects including:
 
**Accuracy:** verifying the correctness of the data.  
**Uniqueness:** identifying and eliminating duplicates.  
**Relevancy:** checking metadata to ensure data is pertinent to the analysis.  
**Validity:** verifying whether data contains allowable values.  
**Consistency:** ensuring values are free from contradiction.  
**Currency:** confirming that data is up to date.  
**Miscellaneous issues:** encompassing any other factors that may impact data accuracy, such as lack of uniformity, etc. 

In [ ]:
# amend column data types
transactions['transaction_date'] = transactions['transaction_date'].astype('datetime64')
transactions['online_order'] = transactions['online_order'].astype('boolean')
transactions['list_price'] = transactions['list_price'].astype('float')
transactions['standard_cost'] = transactions['standard_cost'].astype('float')

In [ ]:
# check information for the changes
transactions.info()

In [ ]:
# check for duplicated rows
transactions[transactions.duplicated()]

The output shows there are no duplicated rows in the data.  

In [ ]:
# check unique values of all columns
column_names = transactions.columns

for i in column_names:
  print((i, transactions[i].is_unique))

In [ ]:
# count unique values of all columns
transactions.nunique()

In [ ]:
# check descriptive statistics for the transaction_date
transactions['customer_id'].describe()

In [ ]:
# filterout customer_id's from 1-3500
transactions = transactions[(transactions['customer_id'] >= 1) & (transactions['customer_id'] <= 3500)]
transactions['customer_id'].describe()

In [ ]:
# check descriptive statistics for the transaction_date
transactions['transaction_date'].describe()

In [ ]:
# check unique values in 'online_order' column
transactions['online_order'].unique()

In [ ]:
# check unique values in 'order_status' column
transactions['order_status'].unique()

In [ ]:
#drop all rows where 'order_status' is equal to 'Cancelled',
transactions.drop(transactions[transactions['order_status'] == 'Cancelled'].index, inplace=True)
transactions['order_status'].unique()

In [ ]:
# check unique values in 'brand' column
transactions['brand'].unique()

In [ ]:
# check unique values in 'product_line' column
transactions['product_line'].unique()

In [ ]:
# check unique values in 'product_class' column
transactions['product_class'].unique()

In [ ]:
# capitalize first letter of unique value for consistency
transactions['product_class'] = transactions['product_class'].str.capitalize()
transactions['product_class'].unique()

In [ ]:
# check unique values in 'product_size' column
transactions['product_size'].unique()

In [ ]:
# capitalize first letter of unique value for consistency
transactions['product_size'] = transactions['product_size'].str.capitalize()
transactions['product_size'].unique()

In [ ]:
# check outliers in 'list_price' column
transactions = transactions[transactions['list_price'] > 0]

In [ ]:
# check the data type of 'product_first_sold_date'
transactions['product_first_sold_date'].dtype

In [ ]:
# convert 'product_first_sold_date' column to a datetime format
transactions['product_first_sold_date'] = pd.to_datetime(transactions['product_first_sold_date'], unit='D', origin=pd.Timestamp('1899-12-30'))

In [ ]:
# read the updated 'product_first_sold_date' column
transactions['product_first_sold_date'].head()

**Creating Additional Variables:**

Adding **transaction_year,  transaction_month name, transaction_day, transaction_day_name** columns to to aid in data analysis and quality assessment.

In [ ]:
# change transactions date column into transaction year,month,day,day_name columns
transactions['transaction_year']=transactions['transaction_date'].dt.year
transactions['transaction_month']=transactions['transaction_date'].dt.month_name()
transactions['transaction_day']=transactions['transaction_date'].dt.day
transactions['day_of_the_week']=transactions['transaction_date'].dt.day_name()

In [ ]:
# convert transaction year, day columnes into string types
transactions['transaction_year']=transactions['transaction_year'].astype(str)
transactions['transaction_day']=transactions['transaction_day'].astype(str)

In [ ]:
# add profit column using list proce and standard cost
transactions['profit'] = transactions['list_price'] - transactions['standard_cost']

In [ ]:
# check transactions dataset after cleaning
transactions

In [ ]:
# check info after cleaning
transactions.info()

The transactions dataset  has been thoroughly cleansed and is now in a state where it can be confidently utilized for data analysis.

----------------------------------------------------------------------------------------------------------------------

## 2.0. Customer_demographic dataset<a id='customer-demographic-dataset'></a>

In [ ]:
# read in customer_demographic sheet from file
customer_demographic = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='CustomerDemographic', header=1)

### 2.1. Data Quality Assessment:<a id='customer-demographic-dataset-data-quality-assessment'></a>
**Intital Examination:**

In [ ]:
# check first few rows
customer_demographic.head()

In [ ]:
# check summary statistics
customer_demographic.info()

In [ ]:
# check descriptive statistics 
customer_demographic.describe()

In [ ]:
# generate a detailed report for "customer_demographic" data set using pandas-profiling
customer_demographic_profile = ProfileReport(customer_demographic)
customer_demographic_profile

**Intital Observations:**  

After examining the dataframe, the identified issues are:

**past_3_years_bike_related_purchases,tenure** is stored as object, instaed of numerical format, potentially affecting data analysis.     
**DOB** is stored as object where it should be converted to datetime.      
Multiple columns have missing values that need addressing before analysis:   **last_name,DOB,job_title,job_industry_category,default,tenure**.

Oldest customer born in 1843, identified as 179 years old, showing a lack of consistency.    
Only 2 out of 4000 customers identified as deceased.    
Gender column values: Male(1872), M(1), Female(2037), F/Femal(2), Unisexual(U)(88) lacks consistency.    
Phik correlation shows no correlation between most variables, except for Gender and Job Industry.    

**Features Engineering:** 

The following strategies can be used to handle missing data:  

**Fill missing values with the mean** for continuous variables without outliers.  
**Fill missing values with the median** for continuous variables with outliers.  
**Fill missing values with the mode** for categorical variables.  
**Use forward-fill (ffill) or backward-fill (bfill)** methods to fill missing data.  
**Drop the entire variable** if there are a large number of missing data points.  
**Drop rows containing null values** if they won't significantly impact the analysis.  

In [ ]:
# count missing values in each variable
customer_demographic.isnull().sum()

In [ ]:
# calculate missing value percentage for each column
customer_demographic.isnull().mean() * 100

**Missing Data:** 

Steps to address missing data:

**last_name**: has just 3.1% missing values, so we fill nulls with mode.  
**DOB**: has 2.2% so we fill nulls with mode.  
**job_title**: has 12.6% we will fill these values with mode.  
**job_industry_category**: has 16.4% so we fill nulls with mode.  
**default**: has 7.6% so we fill nulls with mode.  
**tenure**: has 2.2% missing from the column so we will fill it with mean. 

In [ ]:
# drop column if too many missing datapoints
drop_threshold = customer_demographic.shape[0]*0.5
customer_demographic = customer_demographic.dropna(thresh=drop_threshold, how='all', axis='columns').copy()

In [ ]:
# fill missing values in the 'follwoing' columns with the mode value
customer_demographic['last_name'] = customer_demographic['last_name'].fillna(customer_demographic['last_name'].mode()[0])
customer_demographic['DOB'] = customer_demographic['DOB'].fillna(customer_demographic['DOB'].mode()[0])
customer_demographic['job_title'] = customer_demographic['job_title'].fillna(customer_demographic['job_title'].mode()[0])
customer_demographic['job_industry_category'] = customer_demographic['job_industry_category'].fillna(customer_demographic['job_industry_category'].mode()[0])
customer_demographic['default'] = customer_demographic['default'].fillna(customer_demographic['default'].mode()[0])
# fill missing values in the 'tenure' columns with the mean value
customer_demographic['tenure'] = customer_demographic['tenure'].fillna(customer_demographic['tenure'].mean())

In [ ]:
# check results of updated dataframe
transactions.isnull().sum()

Now the dataset is complete, and ready for addressing other data quality issues.

**Addressing Data Quality Issues:**

To ensure data quality, we will examine the dataframe for several aspects including:
 
**Accuracy:** verifying the correctness of the data.  
**Uniqueness:** identifying and eliminating duplicates.  
**Relevancy:** checking metadata to ensure data is pertinent to the analysis.  
**Validity:** verifying whether data contains allowable values.  
**Consistency:** ensuring values are free from contradiction.  
**Currency:** confirming that data is up to date.  
**Miscellaneous issues:** encompassing any other factors that may impact data accuracy, such as lack of uniformity, etc. 

In [ ]:
# amend column data types
customer_demographic['past_3_years_bike_related_purchases']=customer_demographic['past_3_years_bike_related_purchases'].astype('int')
customer_demographic['tenure']=customer_demographic['tenure'].astype('float')
customer_demographic['DOB']=customer_demographic['DOB'].astype('datetime64')

In [ ]:
# check information for the changes
customer_demographic.info()

In [ ]:
# check for duplicated rows
customer_demographic[customer_demographic.duplicated()]

The output shows there are no duplicated rows in the data.  

In [ ]:
# check unique values of all columns
column_names = customer_demographic.columns
for i in column_names:
  print((i, customer_demographic[i].is_unique))

In [ ]:
# count unique values of all columns
customer_demographic.nunique()

In [ ]:
# check unique values of 'customer_id' column
customer_demographic['customer_id'].unique()

In [ ]:
# filterout customer_id's from 1-3500
customer_demographic = customer_demographic[(customer_demographic['customer_id'] >= 1) & (customer_demographic['customer_id'] <= 3500)]
customer_demographic['customer_id'].describe()

In [ ]:
# check unique values of 'first_name' column
customer_demographic['first_name'].unique()

In [ ]:
# check unique values of 'last_name' column
customer_demographic['last_name'].unique()

In [ ]:
# check unique values of 'gender' column
customer_demographic['gender'].unique()

In [ ]:
# count unique values of 'gender' column
customer_demographic['gender'].value_counts()

In [ ]:
# replace inconsistent values with appropriate values
customer_demographic['gender'] = customer_demographic['gender'].replace('F','Female').replace('M','Male').replace('Femal','Female').replace('U','Unspecified')
customer_demographic['gender'].value_counts()

In [ ]:
# drop unspecified values of gender column 
customer_demographic.drop(customer_demographic[customer_demographic['gender'] == 'Unspecified'].index, inplace=True)
customer_demographic

In [ ]:
# check unique values of 'past_3_years_bike_related_purchases' column
customer_demographic['past_3_years_bike_related_purchases'].unique()

In [ ]:
# check unique values of 'DOB' column
customer_demographic['DOB'].describe()

In [ ]:
# replace inconsistent values with appropriate values
customer_demographic['DOB'] = customer_demographic['DOB'].replace('1843-12-21 00:00:00','1943-12-21 00:00:00')
customer_demographic['DOB'].describe()

In [ ]:
# check unique values of 'job_title' column
customer_demographic['job_title'].unique()

In [ ]:
# check unique values of 'job_industry_category' column
customer_demographic['job_industry_category'].unique()

In [ ]:
# count unique values of 'job_industry_category' column
customer_demographic['job_industry_category'].value_counts()

In [ ]:
# check unique values of 'wealth_segment' column
customer_demographic['wealth_segment'].unique()

In [ ]:
# count unique values of 'wealth_segment' column
customer_demographic['wealth_segment'].value_counts()

In [ ]:
# check unique values of 'deceased_indicator' column
customer_demographic['deceased_indicator'].unique()

In [ ]:
# count unique values of 'deceased_indicator' column
customer_demographic['deceased_indicator'].value_counts()

In [ ]:
# drop all rows where 'deceased_indicator' is equal to 'Y',
customer_demographic.drop(customer_demographic[customer_demographic['deceased_indicator'] == 'Y'].index, inplace=True)

In [ ]:
# check unique values of 'deceased_indicator' column
customer_demographic['deceased_indicator'].value_counts()

In [ ]:
# drop default columns
customer_demographic.drop('default',axis=1,inplace=True)

In [ ]:
# check unique values of 'owns_car' column
customer_demographic['owns_car'].unique()

In [ ]:
# count unique values of 'owns_car' column
customer_demographic['owns_car'].value_counts()

In [ ]:
# check unique values of 'tenure' column
customer_demographic['tenure'].unique()

In [ ]:
# count unique values of 'tenure' column
customer_demographic['tenure'].value_counts()

**Creating Additional Variables:**

Adding **age** column and **age_group** to to aid in data analysis and quality assessment.

In [ ]:
# function converts given date to age
def from_dob_to_age(born):
    today = dt.date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [ ]:
# apply function on the DOB column
customer_demographic['age']=customer_demographic['DOB'].apply(lambda x: from_dob_to_age(x))

In [ ]:
# divide age group
Age_group = pd.cut(customer_demographic.age, bins = [20,30,50,70,92], labels= ['20-30','31-50','51-70','71-92'])
customer_demographic.insert(13, 'age_group', Age_group)

In [ ]:
# check customer_demographic dataset after cleaning
customer_demographic

In [ ]:
# check info after cleaning
customer_demographic.info()

The customer_demographic dataset has been thoroughly cleansed and is now in a state where it can be confidently utilized for data analysis.

----------------------------------------------------------------------------------------------------------------------

## 3.0. Customer_address dataset<a id='customer-address-dataset'></a>

In [ ]:
# read in customer_addres sheet from file
customer_address = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='CustomerAddress', header=1)

### 3.1. Data Quality Assessment:<a id='customer-address-dataset-data-quality-assessment'></a>
**Intital Examination:**

In [ ]:
# check first few rows
customer_address.head()

In [ ]:
# check summary statistics
customer_address.info()

In [ ]:
# check descriptive statistics 
customer_address.describe()

In [ ]:
# generate a detailed report for "customer_demographic" data set using pandas-profiling
customer_address_profile = ProfileReport(customer_address)
customer_address_profile

**Intital Observations:**

Based on an initial examination of the dataframe, there seems no issues with the datatypes, missing values.  
Country column values are all "Australia", which is not useful and unnecessary for memory.    
"State" column has unique values of NSW, VIC, New South Wales, and Victoria, which will be clamped to NSW and VIC.    
Interesting correlations include "state" and "property value", more details to follow.  

**Features Engineering:**  

In [ ]:
# count missing values in each variable
customer_address.isnull().sum()

In [ ]:
# calculate missing value percentage for each column
customer_address.isnull().mean() * 100

**Missing Data:** 

No missing data


**Addressing Data Quality Issues:**

To ensure data quality, we will examine the dataframe for several aspects including:
 
**Accuracy:** verifying the correctness of the data.  
**Uniqueness:** identifying and eliminating duplicates.  
**Relevancy:** checking metadata to ensure data is pertinent to the analysis.  
**Validity:** verifying whether data contains allowable values.  
**Consistency:** ensuring values are free from contradiction.  
**Currency:** confirming that data is up to date.  
**Miscellaneous issues:** encompassing any other factors that may impact data accuracy, such as lack of uniformity, etc. 

In [ ]:
# check for duplicated rows
customer_address[customer_address.duplicated()]

There are no duplicated rows in the data.  
we will now check the consistency of the values in each categorical column in the dataset

In [ ]:
# check unique values of all columns
column_names = customer_address.columns

for i in column_names:
  print((i, customer_address[i].is_unique))

In [ ]:
# count unique values of all columns
customer_address.nunique()

In [ ]:
# check unique values in 'customer_id' column
customer_address['customer_id'].unique()

In [ ]:
# filterout customer_id's from 1-3500
customer_address = customer_address[(customer_address['customer_id'] >= 1) & (customer_address['customer_id'] <= 3500)]
customer_address['customer_id'].describe()

In [ ]:
# check unique values in 'address' column
customer_address['address'].unique()

In [ ]:
# check unique values in 'postcode' column
customer_address['postcode'].value_counts()

In [ ]:
# check unique values in 'state' column
customer_address['state'].unique()

In [ ]:
# count unique  values in the 'state' column
customer_address['state'].value_counts()

In [ ]:
# Replace all the misspelt or differently formatted data
customer_address['state'].replace({'New South Wales' : 'NSW', 'Victoria' : 'VIC'}, inplace=True)
customer_address['state'].unique()

In [ ]:
# check unique values in 'country' column
customer_address['country'].unique()

In [ ]:
# check unique values in 'property_valuation' column
customer_address['property_valuation'].unique()

In [ ]:
# check customer_address dataset after cleaning
customer_address

In [ ]:
# check info after cleaning
customer_address.info()

The customer_address dataset has been thoroughly cleansed and is now in a state where it can be confidently utilized for data analysis.


----------------------------------------------------------------------------------------------------------------------

## 4.0. Data Integration<a id='data-integration'></a>

To prepare for the second phase of the project involving exploratory data analysis, we can perform an inner join on the three datasets (customer transactions, demographics, and addresses) using the customer ID primary key column in the first dataset and the corresponding customer ID foreign key columns in the other datasets.

In [ ]:
# join transactions with customer_demographics datasets
transactions_demographics=transactions.merge(customer_demographic,on='customer_id',how='inner')
transactions_demographics.head()

In [ ]:
# merge customer_demographics & transactions with customer_addresses dataset into a dataset named old_customers
old_customers = transactions_demographics.merge(customer_address,on='customer_id',how='inner')

In [ ]:
# check the info of the three datasets together
old_customers.info()

In [ ]:
# check final dataset first few rows
old_customers.head()

In [ ]:
# check summary statistics
old_customers.info()

In [ ]:
# generate a detailed report for "cdta" data set using pandas-profiling
old_customers_profile = ProfileReport(old_customers)
old_customers_profile

Let's export the dataset to a CSV file for easy importing into the next Jupyter notebook.

In [ ]:
# exporting data to csv file
old_customers.to_csv('old_customers.csv')

----------------------------------------------------------------------------------------------------------------------

## 5.0. New customers dataset:<a id='new-customers-dataset'></a>

The new customer dataset will be imported and adjusted for the machine learning model.

In [ ]:
# read in NewCustomerList sheet from file
new_customers = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='NewCustomerList', header=1)

In [ ]:
# check first few rows
new_customers.head()

In [ ]:
# check summary statistics
new_customers.info()

### 2.1. Data Quality Assessment:<a id='new-customers-dataset-data-quality-assessment'></a>
**Intital Examination:**

In [ ]:
# check descriptive statistics 
new_customers.describe()

In [ ]:
# generate a detailed report for "new_customers" data set using pandas-profiling
new_customers_profile = ProfileReport(new_customers)
new_customers_profile

**Intital Observations:**

After examining the dataframe, the identified issues are:  

Multiple columns have missing values that need addressing before analysis: **last_name, DOB, job_title, job_industry_category**     
Unknown columns identified with meaningless metadata, and no value found.

**Features Engineering:**

The following strategies can be used to handle missing data:  

**Fill missing values with the mean** for continuous variables without outliers.  
**Fill missing values with the median** for continuous variables with outliers.  
**Fill missing values with the mode** for categorical variables.  
**Use forward-fill (ffill) or backward-fill (bfill)** methods to fill missing data.  
**Drop the entire variable** if there are a large number of missing data points.  
**Drop rows containing null values** if they won't significantly impact the analysis.  

In [ ]:
# count missing values in each variable
new_customers.isnull().sum()

In [ ]:
# calculate missing value percentage for each column
new_customers.isnull().mean() * 100

**Missing Data:** 

Steps to address missing data:

**job_title, job_industry_category** has 10% and 16% missing data, which will be imputed using the mode, the most common data point in the dataset.   
**last_name, DOB** have less than 3% missing data. Thus, we will remove the rows containing null values for these variables.

In [ ]:
# drop column if too many missing datapoints
drop_threshold = new_customers.shape[0]*0.5
new_customers = new_customers.dropna(thresh=drop_threshold, how='all', axis='columns').copy()

In [ ]:
# fill in the job_title,job_industry_category column with the mode value which is the most repeated value in the column
new_customers['job_title'] = new_customers['job_title'].fillna(new_customers['job_title'].mode()[0])
new_customers['job_industry_category'] = new_customers['job_industry_category'].fillna(new_customers['job_industry_category'].mode()[0])

In [ ]:
# remove rows with null values in other columns
new_customers.dropna(axis=0,inplace=True)

In [ ]:
# check results after removing null values
new_customers.isnull().sum()

**Addressing Data Quality Issues:**

To ensure data quality, we will examine the dataframe for several aspects including:
 
**Accuracy:** verifying the correctness of the data.  
**Uniqueness:** identifying and eliminating duplicates.  
**Relevancy:** checking metadata to ensure data is pertinent to the analysis.  
**Validity:** verifying whether data contains allowable values.  
**Consistency:** ensuring values are free from contradiction.  
**Currency:** confirming that data is up to date.  
**Miscellaneous issues:** encompassing any other factors that may impact data accuracy, such as lack of uniformity, etc.  

In [ ]:
# dropping U from gender
new_customers=new_customers[new_customers.gender!='U']

In [ ]:
# amend column data types
new_customers['DOB'] = new_customers['DOB'].astype('datetime64')

In [ ]:
# check information for the changes
new_customers.info()

In [ ]:
# check for duplicated rows
new_customers[new_customers.duplicated()]

The output shows there are no duplicated rows in the data.  

In [ ]:
# check unique values of all columns
column_names = new_customers.columns

for i in column_names:
  print((i, new_customers[i].is_unique))

In [ ]:
# count unique values of all columns
new_customers.nunique()

In [ ]:
# collecting the categorical columns into list
cat_col=[]
for x in new_customers.dtypes.index:
    if new_customers.dtypes[x]=='object':
        cat_col.append(x)
cat_col

In [ ]:
# check unique values of 'first_name' column
new_customers['first_name'].unique()

In [ ]:
# check unique values of 'last_name' column
new_customers['last_name'].unique()

In [ ]:
# check unique values of 'gender' column
new_customers['gender'].unique()

In [ ]:
# count unique values of 'gender' column
new_customers['gender'].value_counts()

In [ ]:
# check unique values of 'DOB' column
new_customers['DOB'].describe()

In [ ]:
# check unique values of 'job_title' column
new_customers['job_title'].value_counts()

In [ ]:
# check unique values of 'job_industry_category' column
new_customers['job_industry_category'].value_counts()

In [ ]:
# check unique values of 'wealth_segment' column
new_customers['wealth_segment'].unique()

In [ ]:
# count unique values of 'wealth_segment' column
new_customers['wealth_segment'].value_counts()

In [ ]:
# check unique values of 'deceased_indicator' column
new_customers['deceased_indicator'].unique()

In [ ]:
# count unique values of 'deceased_indicator' column
new_customers['deceased_indicator'].value_counts()

In [ ]:
# drop unknown columns, 16-20 columns
new_customers.drop(['Unnamed: 16','Unnamed: 17','Unnamed: 18','Unnamed: 19','Unnamed: 20'],axis=1,inplace=True)

In [ ]:
# check unique values of 'owns_car' column
new_customers['owns_car'].unique()

In [ ]:
# count unique values of 'owns_car' column
new_customers['owns_car'].value_counts()

In [ ]:
# check unique values of 'tenure' column
new_customers['tenure'].unique()

In [ ]:
# count unique values of 'tenure' column
new_customers['tenure'].value_counts()

In [ ]:
# check unique values of 'address' column
new_customers['address'].unique()

In [ ]:
# check unique values of 'postcode' column
new_customers['postcode'].value_counts()

In [ ]:
# check unique values of 'state' column
new_customers['state'].unique()

In [ ]:
# check unique values of 'country' column
new_customers['country'].unique()

In [ ]:
# check unique values of 'property_valuation' column
new_customers['property_valuation'].value_counts()

In [ ]:
# check unique values of 'Rank' column
new_customers['Rank'].value_counts()

In [ ]:
# check unique values of 'Value' column
new_customers['Value'].value_counts()

**Creating Additional Variables:**

Adding **Age** column to to aid in data analysis and quality assessment.

In [ ]:
# function converts given date to age
def from_dob_to_age(born):
    today = dt.date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [ ]:
# apply function on the DOB column
new_customers['age']=new_customers['DOB'].apply(lambda x: from_dob_to_age(x))

In [ ]:
# divide age group
Age_group = pd.cut(new_customers.age, bins = [20,30,50,70,92], labels= ['20-30','31-50','51-70','71-92'])
new_customers.insert(19, 'age_group', Age_group)

In [ ]:
# check new_customers dataset after cleaning
new_customers

In [ ]:
# check info after cleaning
new_customers.info()

The new_customers dataset  has been thoroughly cleansed and is now in a state where it can be confidently utilized for data analysis.

In [ ]:
# exporting our cleaned dataframe to use it in module03
new_customers.to_csv('new_customers.csv')

Cheers,  
Vamsi Krishna Kamatham